In [3]:
import psycopg2 as ppg
import pandas as pd
import numpy as np
import random
import datetime
import time

In [4]:
usa = (-125.6791025,25.4180700649,-66.885417,
                           49.3284551525)
la = ( -119.578941, 32.799580, -114.130814, 35.809120)
    
def in_place(pt, box, long_field='long', lat_field='lat'):
    return (pt[long_field] > box[0]) & \
             (pt[lat_field] > box[1]) & \
             (pt[long_field] < box[2]) & \
             (pt[lat_field] < box[3])

In [ ]:
con = ppg.connect(dbname="tweets", user="patrick")
con.rollback()
with con.cursor() as cur:
    cur.execute("select user_id, place, coded_lat, coded_long "
                "from users join places using (place) where place != 'None' AND place is not null and place != '' AND "
               "coded_lat != -500 AND coded_long != 500")
    user_df = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

In [ ]:
with con.cursor() as cur:
    cur.execute("select tweet_id, lat, long from tweets")
    tweet_loc_df = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])
tweet_loc_df['in_us'] = tweet_loc_df.apply(lambda x: in_place(x, usa), axis=1)